In [ ]:
import pandas as pd
import folium
import json
import branca.colormap as cm

In [ ]:
dfEstaciones =pd.read_csv('../CSVs/station.csv', thousands=',', low_memory=False)
dfViajes = pd.read_csv('../CSVs/trip.csv', thousands=',', low_memory=False)

In [ ]:
latPromedio = dfEstaciones.lat.mean()
longPromedio = dfEstaciones.long.mean()

In [ ]:
dfEstaciones

In [ ]:
dfViajes

In [ ]:
cantidadTotalDeBicicletasSoportadas = dfEstaciones.dock_count.sum()
print cantidadTotalDeBicicletasSoportadas

In [ ]:
map1 = folium.Map(location=[latPromedio, longPromedio], tiles = "Stamen Terrain")
map1Copy = folium.Map(location=[latPromedio, longPromedio], tiles = "Stamen Terrain")
map2 = folium.Map(location=[latPromedio, longPromedio], tiles = "Stamen Terrain")
startStationMap = folium.Map(location=[latPromedio, longPromedio], tiles = "Stamen Terrain")
viajesEntreEstacionesMap = folium.Map(location=[latPromedio, longPromedio], tiles = "Stamen Terrain")


In [ ]:
grupoDeUbicaciones = folium.FeatureGroup(name='Ubicación de estaciones 1')
for lat, lon, nombre in zip(dfEstaciones.lat, dfEstaciones.long, dfEstaciones.name):
    grupoDeUbicaciones.add_child(folium.Marker(location=[lat,lon], popup=nombre))
    
map1.add_child(grupoDeUbicaciones)

In [ ]:
# Ahora intento version con MarkerCluster
# map1Copy.add_child(folium.MarkerCluster(locations=list(zip(dfEstaciones.long, dfEstaciones.lat))))

clusterDeUbicaciones = folium.MarkerCluster().add_to(map1Copy)
for lat, lon, nombre in zip(dfEstaciones.lat, dfEstaciones.long, dfEstaciones.name):
    folium.Marker(location=[lat,lon], popup=nombre).add_to(clusterDeUbicaciones)
    
# map1.add_child(grupoDeUbicaciones)
map1Copy

In [ ]:
grupoDeUbicacionesConCantidades = folium.FeatureGroup(name='Ubicación de estaciones')
for lat, lon, nombre, cantidadDeBicicletasSoportadas in zip(dfEstaciones.lat, dfEstaciones.long, dfEstaciones.name, dfEstaciones.dock_count):
    grupoDeUbicacionesConCantidades.add_child(folium.CircleMarker(location=[lat,lon], popup=nombre, 
                                                     radius=cantidadDeBicicletasSoportadas, 
                                                     fill_color='#b3ffb3', color='#00ff00'))
    
map2.add_child(grupoDeUbicacionesConCantidades)

In [ ]:
dfStartStationCount = dfViajes[['id', 'start_station_id']].groupby(by='start_station_id', as_index=False)\
                                                .count().rename(columns={'id': 'cantidad_de_partidas'})
dfStartStationCount

In [ ]:
dfPartidas = dfEstaciones[['id', 'lat', 'long', 'name']].merge(pd.DataFrame(dfStartStationCount), \
         left_on='id', right_on='start_station_id', how='right')
dfPartidas

In [ ]:
cantidad_de_partidas_maxima = dfPartidas.cantidad_de_partidas.max()
cantidad_de_partidas_maxima

In [ ]:
grupoDeUbicacionesConCantidadesDePartidas = folium.FeatureGroup(name='Ubicación de estaciones con infor de #partidas')
for lat, lon, nombre, cant in zip(dfPartidas.lat, dfPartidas.long, dfPartidas.name, dfPartidas.cantidad_de_partidas):
    grupoDeUbicacionesConCantidadesDePartidas.add_child(folium.CircleMarker(location=[lat,lon], popup=nombre, 
                                                     radius=cant/1000,
                                                     fill_color='#8A2BE2', color='#00ff00'))
    
startStationMap.add_child(grupoDeUbicacionesConCantidadesDePartidas)

In [ ]:
# df = pd.DataFrame(columns=['start_station_id', 'start_station_lat', 'start_station_lon',
#                            'end_station_id', 'end_station_lat', 'end_station_lon'])
# df.start_station_id = dfViajes.start_station_id
# df
# dfStartStationCount['start_station_lat'] = dfEstaciones['lat']
# dfStartStationCount['start_station_lon'] = dfEstaciones['long']
# dfStartStationCount['name'] = dfEstaciones['name']
# for idEstacionSalida in dfStartStationCount.index.tolist():
#     dfStartStationCount['start_station_lat'] = dfEstaciones.loc[dfEstaciones.id == idEstacionSalida, 'lat'] 
# dfStartStationCount

In [ ]:
# grupoDeUbicacionesConCantidades = folium.FeatureGroup(name='Ubicación de estaciones')
# for lat, lon, nombre, cantidadDeBicicletasSoportadas in zip(dfEstaciones.lat, dfEstaciones.long, dfEstaciones.name, dfEstaciones.dock_count):
#     grupoDeUbicacionesConCantidades.add_child(folium.CircleMarker(location=[lat,lon], popup=nombre, 
#                                                      radius=cantidadDeBicicletasSoportadas, 
#                                                      fill_color='#b3ffb3', color='#00ff00'))
    
# map2.add_child(grupoDeUbicacionesConCantidades)

In [ ]:
dfViajesEntreEstaciones = dfViajes[['id', 'start_station_id', 'end_station_id']]\
        .groupby(by=['start_station_id', 'end_station_id'], as_index=False)\
        .count().rename(columns={'id': 'cantidad'})
dfViajesEntreEstaciones

In [ ]:
dfViajesEntreEstaciones = dfViajesEntreEstaciones.merge(right=dfEstaciones[['lat', 'long', 'id', 'name']], \
                              left_on='start_station_id', right_on='id')\
                            .drop(['id'], 1).rename(columns={'lat': 'start_station_lat', 'long': \
                                                          'start_station_long', 'name': 'start_station_name'})

In [ ]:
dfViajesEntreEstaciones = dfViajesEntreEstaciones.merge(right=dfEstaciones[['lat', 'long', 'id', 'name']], \
                              left_on='end_station_id', right_on='id')\
                            .drop(['id'], 1).rename(columns={'lat': 'end_station_lat', 'long': \
                                                          'end_station_long', 'name': 'end_station_name'})
dfViajesEntreEstaciones.sort_values(by='cantidad', ascending=True)

In [ ]:
dfViajesEntreEstaciones.cantidad.describe()

In [ ]:
maxCantidadDeViajes = dfViajesEntreEstaciones.cantidad.max()
promedioCantidadDeViajes = dfViajesEntreEstaciones.cantidad.mean()
dfViajesEntreEstaciones.cantidad.max()

In [ ]:
grupoDeViajesEntreEstaciones = folium.FeatureGroup(name='Viajes entre estaciones')
for startLat, startLon, startName, endLat, endLon, endName, cant in zip(dfViajesEntreEstaciones.start_station_lat\
                                                                       dfViajesEntreEstaciones.start_station_long\
                                                                       dfViajesEntreEstaciones.start_station_name\
                                                                       dfViajesEntreEstaciones.end_station_lat\
                                                                       dfViajesEntreEstaciones.end_station_long\
                                                                       dfViajesEntreEstaciones.end_station_name\
                                                                       dfViajesEntreEstaciones.cantidad):
    grupoDeUbicacionesConCantidadesDePartidas.add_child(folium.Marker(location=[lat,lon], popup=nombre, 
                                                     radius=cant/1000,
                                                     fill_color='#8A2BE2', color='#00ff00'))
    
startStationMap.add_child(grupoDeUbicacionesConCantidadesDePartidas)

In [ ]:
# colores = cm.LinearColormap(['green', 'orange', 'red'], vmax=maxCantidadDeViajes, 
#                   index=[0, promedioCantidadDeViajes, maxCantidadDeViajes])

# colores = cm.LinearColormap(['red', 'orange', 'yellow', 'green'], vmax = maxCantidadDeViajes)

colores = cm.StepColormap(['green', 'yellow', 'magenta', 'red'],
                       vmin=1, vmax=maxCantidadDeViajes, index=[1, 71.5, 191, 440, maxCantidadDeViajes],
                       caption='step')
colores.caption = 'Cantidad de viajes entre estaciones'
# colores = cm.linear.Set1.scale(0, maxCantidadDeViajes).to_step(
# #     n=4,
#     index=[1, 71.5, 191, 440, 6216],
# #     method='quantiles',
# #     round_method='int'
# )
# colores.to_linear()
colores2 = cm.linear.YlGn.scale(0, 6000)
colores3 = cm.linear.RdGy.to_step(100).scale(0, 6000)


colores2

In [ ]:
linesMap = folium.Map(location=[latPromedio, longPromedio], tiles = "Stamen Terrain")
linesMap.add_child(colores3)

In [ ]:
clusterDeUbicaciones = folium.MarkerCluster().add_to(linesMap)
for lat, lon, nombre in zip(dfEstaciones.lat, dfEstaciones.long, dfEstaciones.name):
    folium.Marker(location=[lat,lon], popup=nombre).add_to(clusterDeUbicaciones)

In [ ]:
for startLat, startLon, startName, endLat, endLon, endName, cant in zip(dfViajesEntreEstaciones.start_station_lat,\
                                                                       dfViajesEntreEstaciones.start_station_long,\
                                                                       dfViajesEntreEstaciones.start_station_name,\
                                                                       dfViajesEntreEstaciones.end_station_lat,\
                                                                       dfViajesEntreEstaciones.end_station_long,\
                                                                       dfViajesEntreEstaciones.end_station_name,\
                                                                       dfViajesEntreEstaciones.cantidad):
    folium.PolyLine(locations=[(startLat, startLon), (endLat, endLon)], color=colores3(cant), weight=3, opacity=0.001).add_to(linesMap)
#     linesMap.add_child(c)
    
# startStationMap.add_child(grupoDeViajesEntreEstaciones)
linesMap

In [ ]:
lat = +38.89399
lon = -77.03659
zoom_start = 17

m = folium.Map(location=[lat, lon], zoom_start=zoom_start)

kw = dict(fill_color='red', radius=20)
c0 = folium.CircleMarker([38.89415, -77.03738], **kw)
c1 = folium.CircleMarker([38.89415, -77.03578], **kw)


locations = [[(38.893596444352134, -77.03814983367920), (38.893379333722040, -77.03792452812195)],
             [(38.893379333722040, -77.03792452812195), (38.893162222428310, -77.03761339187622)],
             [(38.893162222428310, -77.03761339187622), (38.893028615148424, -77.03731298446655)],
             [(38.893028615148424, -77.03731298446655), (38.892920059048464, -77.03691601753235)],
             [(38.892920059048464, -77.03691601753235), (38.892903358095296, -77.03637957572937)],
             [(38.892903358095296, -77.03637957572937), (38.893011914220770, -77.03592896461487)],
             [(38.893011914220770, -77.03592896461487), (38.893162222428310, -77.03549981117249)],
             [(38.893162222428310, -77.03549981117249), (38.893404384982480, -77.03514575958252)],
             [(38.893404384982480, -77.03514575958252), (38.893596444352134, -77.03496336936950)]]

c3 = folium.PolyLine(locations=locations, color='orange', weight=30, opacity=1)

for c in [c0, c1, c3]:
    m.add_child(c)

m